# Ambiente de trabajo - Hunt the wumpus!

In [372]:
# define rutas de trabajo e importa ambiente de trabajo
import os
import sys

CURRENT_PATH = os.getcwd()
NOTEBOOKS_PATH = os.path.join(CURRENT_PATH, os.path.pardir, 'notebooks')
PYTHON_PATH = os.path.join(CURRENT_PATH, os.path.pardir, 'python')
sys.path.append(PYTHON_PATH)

from logica import *
from entornos import *
from wkb import *
from time import sleep
from IPython.display import clear_output
import matplotlib.pyplot as plt

from busqueda import best_first_search, solucion, breadth_first_search



In [373]:
# Crea el objeto agente con la base de conocimientos para transitar seguro y salir de la cueva

W = Wumpus(wumpus=(3,3), oro=(1,2), pozos=[(0,3), (2,2)])
agente = Agente_wkb()
formulas = agente.fluentes_mapa_mental_wkb()
formulas += agente.brisa_pozo_wkb()
formulas += agente.hedor_wumpus_wkb()
formulas += agente.casilla_segura_wkb()
# formulas += agente.wumpus_muerto() # agregar: ya no importa el hedor del wumpus
# formulas += agente.caza_wumpus() # agregar: 
formulas += [f'en(0,0)_1', 'mirando_e_1', 'segura(0,0)']
agente.base = LPQuery(formulas)
agente.perceptos = W.para_sentidos()
c = agente.interp_percepto_wkb(mundo='wumpus')
agente.base.TELL(c)


## Programa agente

La función `programaKB`, que es un método de la clase agente, es la función a modificar y en ella debe incluirse la base de conocimiento necesaria para cazar al wumpus.

In [199]:
def programaKB(self, DEB=False):
    acciones = []
    turno, casilla_actual, direccion, cas_seguras, cas_visitadas = self.cache_wkb()

    if DEB:
        print('')
        print('Turno acutal:', turno)
        print('Casilla actual:', casilla_actual)
        print('Dirección actual:', direccion)
        print('Casillas adyacentes seguras:', cas_seguras)
        print('Casillas visitadas:', cas_visitadas)
        print('¿El oro ha sido tomado?: ', W.oro_tomado)
        print('¿El wumpus sigue con vida?: ', W.wumpus_vivo)

    if ASK(f'brillo_{turno}','success',self.base):
        if DEB:
            print('¡Oh, el oro!')
        acciones.append('agarrar')
        R = Rejilla(casilla_actual, (0,0), self.todas_seguras_wkb())
        camino = best_first_search(R)
        camino = [casilla_actual] + solucion(camino)
        acciones += acciones_camino(camino, direccion)
        acciones.append('salir')
    else:
        opciones = [casilla for casilla in cas_seguras if casilla not in cas_visitadas]
        if DEB:
            print('Casillas opcionales:', opciones)
        if len(opciones) > 0:
            casilla_ir = choice(opciones)
            if DEB:
                print('El agente quiere ir a la casilla', casilla_ir)
            camino = [casilla_actual, casilla_ir]
            acciones = acciones_camino(camino, direccion)
        elif len(cas_seguras) > 0:
            casilla_ir = choice(cas_seguras)
            if DEB:
                print('El agente quiere devolverse a la casilla', casilla_ir)
            camino = [casilla_actual, casilla_ir]
            acciones = acciones_camino(camino, direccion)
        else:
            print("¡Caso no contemplado!")
    self.acciones += acciones

setattr(Agente_wkb, 'programa', programaKB)

## ¡Jugar!

In [200]:
# ejecuta el juego 

for i in range(10):
    a = agente.reaccionar_wkb(DEB=True) # DEB = False by default
    formula_accion = f'{a}_{agente.turno-1}'
    agente.base.TELL(formula_accion)
    W.transicion(a)
    if not W.juego_activo:
        clear_output(wait=True)
        W.pintar_todo()
        print(W.mensaje)
        break
    agente.estimar_estado_wkb(W)
    #clear_output(wait=True)
    #W.pintar_todo()
    #plt.show()
    #sleep(.3) 


Turno acutal: 1
Casilla actual: (0, 0)
Dirección actual: este
Casillas adyacentes seguras: [(1, 0), (0, 1)]
Casillas visitadas: [(0, 0)]
¿El oro ha sido tomado?:  False
¿El wumpus sigue con vida?:  True
Casillas opcionales: [(1, 0), (0, 1)]
El agente quiere ir a la casilla (1, 0)

Turno acutal: 2
Casilla actual: (1, 0)
Dirección actual: este
Casillas adyacentes seguras: [(0, 0), (2, 0), (1, 1)]
Casillas visitadas: [(0, 0), (1, 0)]
¿El oro ha sido tomado?:  False
¿El wumpus sigue con vida?:  True
Casillas opcionales: [(2, 0), (1, 1)]
El agente quiere ir a la casilla (2, 0)

Turno acutal: 3
Casilla actual: (2, 0)
Dirección actual: este
Casillas adyacentes seguras: [(1, 0), (3, 0), (2, 1)]
Casillas visitadas: [(0, 0), (1, 0), (2, 0)]
¿El oro ha sido tomado?:  False
¿El wumpus sigue con vida?:  True
Casillas opcionales: [(3, 0), (2, 1)]
El agente quiere ir a la casilla (2, 1)

Turno acutal: 5
Casilla actual: (2, 1)
Dirección actual: norte
Casillas adyacentes seguras: [(1, 1), (2, 0)]
Casi